In [93]:
using LinearAlgebra, CSV, Random, Tables, Tulip, MathOptInterface, SparseArrays

In [30]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
k = 7;
# All close.
maximum(abs.(raw_data * raw_data .- data))

7.327471962526033e-15

In [31]:
nodes = CSV.File("../NC-K7-Trace-Nodes.csv") |> Tables.matrix;
bounds = CSV.File("../NC-K7-Trace-Bounds.csv") |> Tables.matrix;

In [40]:
# node_id = 50
node_id = 300
y = nodes[:, node_id]
sum(nodes[:, node_id] .== 1)

3

In [33]:
bounds[:, node_id]

3-element Array{Float64,1}:
 3.608137513865839
 6.26741838326191
 7.0

In [34]:
selected_data = findall(nodes[:, node_id] .== 1)

3-element Array{Int64,1}:
 58
 60
 62

In [35]:
s = maximum(svd(raw_data[:, selected_data]).S)
u = svd(raw_data[:, selected_data]).U[:, 1];
v = svd(raw_data[:, selected_data]).V[:, 1];
D = Matrix(1.0I, 101, 101)[:, selected_data];
s

1.664966558927097

In [36]:
rank_one_term = (raw_data * u) .^ 2
rank_one_term[nodes[:, node_id] .== 0] .= 0
rank_one_term

101-element Array{Float64,1}:
 0.042608386423828616
 0.12169366148453947
 0.0010827657107647158
 0.0
 0.3576958952379637
 0.5909313328874289
 0.002813434428971572
 0.045710787852979134
 0.013475878738447642
 0.04025071431940448
 ⋮
 0.19651099861244614
 0.035215018953541156
 0.004623427695183872
 0.026081129247667843
 0.0008624637095038491
 0.37524469637807856
 0.11466762911354499
 0.03909959370980694
 0.0

In [37]:
function frobenius_rows_k(M, y)
    M = M .^ 2
    row_quantities = zeros(n)
    stillneed = k - sum(y .== 1)
    for row_index in 1:n
        if y[row_index] == 0
            continue
        end
        row = copy(M[row_index, :])
        row_sum = sum(row[y .== 1])
        row[y .== 1] .= 0
        if y[row_index] == -1
            row_sum += row[row_index]
            row[row_index] = 0
            row_stillneed = stillneed -1
        else
            row_stillneed = stillneed
        end
        row_sum += sum(sort(row, rev=true)[1:row_stillneed])
        row_quantities[row_index] = row_sum
    end
    row_quantities
end

frobenius_rows_k (generic function with 1 method)

In [41]:
numer = frobenius_rows_k(data^2, y)

101-element Array{Float64,1}:
  427.03738335285755
  468.86353577814776
  642.5059792688386
    0.0
  689.6115085232227
 1073.5283830910835
  267.83972592428563
  447.5689092631501
  308.0308179620631
  352.63135124827835
    ⋮
  401.6730147819353
  290.3973073470707
  140.393722730808
  147.37145422257612
  123.39145560752425
  588.9756265371327
  270.70322612705627
  212.8456812706541
    0.0

In [123]:
optim = Tulip.Optimizer()
d = MathOptInterface.add_variables(optim, n)

101-element Array{MathOptInterface.VariableIndex,1}:
 MathOptInterface.VariableIndex(1)
 MathOptInterface.VariableIndex(2)
 MathOptInterface.VariableIndex(3)
 MathOptInterface.VariableIndex(4)
 MathOptInterface.VariableIndex(5)
 MathOptInterface.VariableIndex(6)
 MathOptInterface.VariableIndex(7)
 MathOptInterface.VariableIndex(8)
 MathOptInterface.VariableIndex(9)
 MathOptInterface.VariableIndex(10)
 ⋮
 MathOptInterface.VariableIndex(93)
 MathOptInterface.VariableIndex(94)
 MathOptInterface.VariableIndex(95)
 MathOptInterface.VariableIndex(96)
 MathOptInterface.VariableIndex(97)
 MathOptInterface.VariableIndex(98)
 MathOptInterface.VariableIndex(99)
 MathOptInterface.VariableIndex(100)
 MathOptInterface.VariableIndex(101)

In [124]:
MathOptInterface.set(
    optim,
    MathOptInterface.ObjectiveFunction{MathOptInterface.ScalarAffineFunction{Float64}}(),
    MathOptInterface.ScalarAffineFunction(
        MathOptInterface.ScalarAffineTerm.(numer, d), 0.0,
    ),
)
MathOptInterface.set(optim, MathOptInterface.ObjectiveSense(), MathOptInterface.MAX_SENSE)

In [125]:
MathOptInterface.ScalarAffineFunction(
    MathOptInterface.ScalarAffineTerm.(Float64(1)/k * ones(n), d), 0.0,
)

MathOptInterface.ScalarAffineFunction{Float64}(MathOptInterface.ScalarAffineTerm{Float64}[MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(1)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(2)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(3)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(4)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(5)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(6)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(7)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(8)), MathOptInterface.ScalarAffineTerm{Float64}(0.14285714285714285, MathOptInterface.VariableIndex(9)), MathOptInt

In [126]:
Matrix(sparse([5], [1], [1.], n, 1))[:, 1]

101-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [127]:
for d_elem in d
    indicator = Matrix(sparse([d_elem.value], [1], [1.], n, 1))[:, 1]
    MathOptInterface.add_constraint(
        optim,
        d_elem,
        MathOptInterface.GreaterThan(0.),
    )
    MathOptInterface.add_constraint(
        optim,
        MathOptInterface.ScalarAffineFunction(
            MathOptInterface.ScalarAffineTerm.(indicator .- Float64(1)/k * ones(n), d), 0.0,
        ),
        MathOptInterface.LessThan(0.),
    )
end

In [128]:
MathOptInterface.add_constraint(
    optim,
    MathOptInterface.ScalarAffineFunction(
        MathOptInterface.ScalarAffineTerm.(rank_one_term, d), 0.0,
    ),
    MathOptInterface.EqualTo(1.),
)

MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}}(102)

In [129]:
MathOptInterface.optimize!(optim)

In [130]:
MathOptInterface.get(optim, MathOptInterface.TerminationStatus())

OPTIMAL::TerminationStatusCode = 1

In [131]:
sqrt(MathOptInterface.get(optim, MathOptInterface.ObjectiveValue()))

7778.219390564368